In [26]:
!pip install pandas
!pip install google-analytics-data
!pip install matplotlib
!pip install seaborn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached contourpy-1.3.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.7-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached pillow-11.0.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 2.5 MB/s eta 0:00:00a 0:00:01
Using cached contourpy-1.3.1-cp313-cp313-macosx_11_0_arm64.whl (255 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 4.3 MB/s eta 0:00:00a 0:00:01
Using cached kiwisolver-1.4.7-cp313-cp313-macosx_11_0_arm64.whl (63 kB)
Using cached pillow-11.0.0-cp313-

In [27]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from datetime import datetime, timedelta
from google.analytics.data_v1beta.types import (DateRange, Dimension, Metric, RunReportRequest)
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/mac/Downloads/Redshift integration-bdb44d4849b7.json"

# Налаштування відображення pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [150]:
def fetch_paginated_data(client, request, page_size=10000):
    rows = []
    current_offset = 0
    
    while True:
        request.offset = current_offset
        request.limit = page_size
        
        response = client.run_report(request)
        rows.extend(response.rows)
        
        if len(response.rows) < page_size: break
        
        current_offset += page_size
    
    return rows

In [95]:
def extract_ad_params(url):
    params = {
        'wbraid': None,
        'gbraid': None,
        'gclid': None
    }
    
    if not url or not isinstance(url, str):
        return params
    
    url_parts = url.split('&')
    for part in url_parts:
        for param in params.keys():
            if f"{param}=" in part:
                try:
                    value = part.split(f"{param}=")[1]
                    params[param] = value
                except:
                    continue
                
    return params

In [159]:
def get_ga4_data(property_id: str, days_ago: int = 30, filter_ads: int = 0):
    client = BetaAnalyticsDataClient()
    
    try:
        # Запит 1: Основні метрики
        request1 = RunReportRequest(
            property=f"properties/{property_id}",
            date_ranges=[DateRange(start_date=f"{days_ago}daysAgo", end_date="today")],
            dimensions=[
                Dimension(name=d) for d in [
                    "date", "sessionCampaignId", "sessionCampaignName",
                    "sessionSourceMedium", "country", "deviceCategory", "operatingSystemWithVersion", "platform"
                ]
            ],
            metrics=[
                Metric(name=m) for m in ["sessions", "publisherAdImpressions", "publisherAdClicks", "totalRevenue", "screenPageViews", "bounceRate"]
            ]
        )

        response1_rows = fetch_paginated_data(client, request1)

        # Запит 2: Дані про сторінки
        request2 = RunReportRequest(
            property=f"properties/{property_id}",
            date_ranges=[DateRange(start_date=f"{days_ago}daysAgo", end_date="today")],
            dimensions=[
                Dimension(name=d) for d in [
                    "date", "sessionCampaignId", "sessionSourceMedium",
                    "country", "pagePath", "landingPage", "pageReferrer", "platform"
                ]
            ],
            metrics=[
                Metric(name=m) for m in ["screenPageViews", "bounceRate"]
            ]
        )

        response2_rows = fetch_paginated_data(client, request2)


        # Перевіряємо, чи є дані
        if not response1_rows or not response2_rows:
            print(f"No data found for property {property_id}")
            return pd.DataFrame()

        # Обробка першого набору даних
        data1 = []
        for row in response1_rows:
            d = row.dimension_values
            m = row.metric_values
            data1.append({
                'date': pd.to_datetime(d[0].value).strftime('%Y-%m-%d'),
                'campaign_id': d[1].value,
                'campaign_name': d[2].value,
                'source_medium': d[3].value,
                'country': d[4].value,
                'device_type': d[5].value,
                'os': d[6].value,
                'platform': d[7].value,
                'sessions': int(float(m[0].value)),
                'impressions': int(float(m[1].value)),
                'clicks': int(float(m[2].value)),
                'revenue': float(m[3].value)
            })

        # Обробка другого набору даних
        data2 = []
        for row in response2_rows:
            d = row.dimension_values
            m = row.metric_values
            ad_params = extract_ad_params(d[6].value)  # Витягуємо параметри з referrer
            data2.append({
                'date': pd.to_datetime(d[0].value).strftime('%Y-%m-%d'),
                'campaign_id': d[1].value,
                'source_medium': d[2].value,
                'country': d[3].value,
                'page_path': d[4].value,
                'landing_page': d[5].value,
                'referrer': d[6].value,
                'platform': d[7].value,
                'wbraid': ad_params['wbraid'],
                'gbraid': ad_params['gbraid'],
                'gclid': ad_params['gclid'],
                'page_views': int(float(m[0].value)),
                'bounce_rate': float(m[1].value)
            })

        df1 = pd.DataFrame(data1).drop_duplicates()
        df2 = pd.DataFrame(data2).drop_duplicates()
        print(len(df1), len(df2))

        df3 = pd.merge(df1, df2, 
                       on=['date', 'campaign_id', 'source_medium', 'country', 'platform'], 
                       how='inner')

        # Видаляємо дублікати колонок, якщо є
        df3 = df3.loc[:, ~df3.columns.duplicated()]

        # Фільтруємо за рекламними параметрами, якщо потрібно
        if filter_ads == 1:
            df3 = df3[df3[['wbraid', 'gbraid', 'gclid']].notna().any(axis=1)]

        count_ad_rows = df3[['wbraid', 'gbraid', 'gclid']].notna().any(axis=1).sum()
        print(f"Rows processed: \t\t\t{len(df3):,}".replace(',', '_') )
        print(f"Rows with id (wbraid, gbraid, gclid): \t{count_ad_rows:,}".replace(',', '_'))
        print(f"Percentage: \t\t\t\t{int(count_ad_rows/len(df3)*1000)/10}%\n")

        return df3

    except Exception as e:
        print(f"Error accessing GA4: {str(e)}\n")
        raise

In [160]:
sites_data = {
    # "online-dating-review.net": 450191495,
    "avodate.com": 350536871,
    "datempire.com": 358067421,
    # "feelflame.com": 358106858,
    # "latidate.com": 358050088,
    # "myspecialdates.com": 322504563,
    # "okamour.com": 350538354,
    # "sakuradate.com": 358590047,
    # "sofiadate.com": 322587243,
    "loveforheart.com": 322569296
}

for domain, ga_id in sites_data.items():
    try:
        print(f"Domain: {domain} ")
        df = get_ga4_data(str(ga_id), days_ago=3, filter_ads=0)
        df.to_csv(f"result/ga4_data_{domain.split('.')[0]}_wide.csv", index=False)
    except Exception as e:
        print(f"Error with {domain}: {e}")

Domain: avodate.com 
538 16624
Rows processed: 			186_529
Rows with id (wbraid_ gbraid_ gclid): 	3_590
Percentage: 				1.9%

Domain: datempire.com 


KeyboardInterrupt: 

In [167]:
import time
def get_ga4_data_uno(property_id: str, days_ago: int = 30, filter_ads: int = 0):
    start_time = time.time()
    client = BetaAnalyticsDataClient()
    
    try:
        request2 = RunReportRequest(
            property=f"properties/{property_id}",
            date_ranges=[DateRange(start_date=f"{days_ago}daysAgo", end_date="today")],
            dimensions=[
                Dimension(name=d) for d in [
                    "date", "sessionCampaignId", "sessionSourceMedium",
                    "country", "pagePath", "landingPage", "pageReferrer", "platform"
                ]
            ],
            metrics=[
                Metric(name=m) for m in ["sessions", "publisherAdImpressions", "publisherAdClicks", "totalRevenue", "screenPageViews", "bounceRate"]
            ]
        )

        response2_rows = fetch_paginated_data(client, request2)


        # Перевіряємо, чи є дані
        if not response2_rows :
            print(f"No data found for property {property_id}")
            return pd.DataFrame()


        # Обробка другого набору даних
        data2 = []
        for row in response2_rows:
            d = row.dimension_values
            m = row.metric_values
            ad_params = extract_ad_params(d[6].value)  # Витягуємо параметри з referrer
            data2.append({
                'date': pd.to_datetime(d[0].value).strftime('%Y-%m-%d'),
                'campaign_id': d[1].value,
                'source_medium': d[2].value,
                'country': d[3].value,
                'page_path': d[4].value,
                'landing_page': d[5].value,
                'referrer': d[6].value,
                'platform': d[7].value,
                'wbraid': ad_params['wbraid'],
                'gbraid': ad_params['gbraid'],
                'gclid': ad_params['gclid'],
                'page_views': int(float(m[4].value)),
                'bounce_rate': float(m[5].value),
                'publisher_ad_impressions': float(m[1].value),
                'publisher_ad_clicks': float(m[2].value),
                'total_revenue': float(m[3].value)
            })

        df3 = pd.DataFrame(data2).drop_duplicates()

        df3 = df3.loc[:, ~df3.columns.duplicated()]

        # Фільтруємо за рекламними параметрами, якщо потрібно
        if filter_ads == 1:
            df3 = df3[df3[['wbraid', 'gbraid', 'gclid']].notna().any(axis=1)]

        count_ad_rows = df3[['wbraid', 'gbraid', 'gclid']].notna().any(axis=1).sum()
        print(f"Rows processed: \t\t\t{len(df3):,}".replace(',', '_') )
        print(f"Rows with id (wbraid, gbraid, gclid): \t{count_ad_rows:,}".replace(',', '_'))
        print(f"Percentage: \t\t\t\t{int(count_ad_rows/len(df3)*1000)/10}%\n")
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Function executed in {elapsed_time:.2f} seconds")
        return df3

    except Exception as e:
        print(f"Error accessing GA4: {str(e)}\n")
        raise

    

In [ ]:
sites_data = {
    # "online-dating-review.net": 450191495,
    "avodate.com": 350536871,
    "datempire.com": 358067421,
    "feelflame.com": 358106858,
    "latidate.com": 358050088,
    "myspecialdates.com": 322504563,
    "okamour.com": 350538354,
    "sakuradate.com": 358590047,
    "sofiadate.com": 322587243,
    "loveforheart.com": 322569296
}

for domain, ga_id in sites_data.items():
    try:
        print(f"Domain: {domain} ")
        df = get_ga4_data_uno(str(ga_id), days_ago=10, filter_ads=0)
        df.to_csv(f"result/ga4_data_{domain.split('.')[0]}_wide.csv", index=False)
    except Exception as e:
        print(f"Error with {domain}: {e}")

Domain: avodate.com 
Rows processed: 			62_272
Rows with id (wbraid_ gbraid_ gclid): 	1_115
Percentage: 				1.7%

Function get_ga4_data executed in 26.31 seconds
Domain: datempire.com 
Rows processed: 			135_112
Rows with id (wbraid_ gbraid_ gclid): 	30_992
Percentage: 				22.9%

Function get_ga4_data executed in 69.23 seconds
Domain: feelflame.com 
Rows processed: 			44_788
Rows with id (wbraid_ gbraid_ gclid): 	13_869
Percentage: 				30.9%

Function get_ga4_data executed in 21.27 seconds
Domain: latidate.com 
Rows processed: 			169_903
Rows with id (wbraid_ gbraid_ gclid): 	18_582
Percentage: 				10.9%

Function get_ga4_data executed in 81.72 seconds
Domain: myspecialdates.com 
Rows processed: 			644_431
Rows with id (wbraid_ gbraid_ gclid): 	120_532
Percentage: 				18.7%

Function get_ga4_data executed in 382.00 seconds
Domain: okamour.com 
